In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pandas as pd
import time

In [ ]:
# Load environment variables from a .env file
load_dotenv()

# # Set up your Spotify API credentials
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Get audio features from Spotify

In [26]:
# Reading your DataFrame
songs_df = pd.read_csv('songs_with_lyrics_clean.csv')
# Drop rows where 'valence' is null
songs_df.dropna(subset=['lyrics'], inplace=True)
print(songs_df.shape)

# Add columns for valence and energy
songs_df['valence'] = None
songs_df['energy'] = None

(10517, 9)


In [ ]:
# Function to get Spotify access token
def get_spotify_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    
    try:
        auth_response = requests.post(auth_url, {
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret,
        })
        auth_response.raise_for_status()
        
        auth_response_data = auth_response.json()
        return auth_response_data['access_token']
    
    except requests.exceptions.RequestException as e:
        print(f"Error getting Spotify token: {e}")
        return None

# Getting the access token
token = get_spotify_token(client_id, client_secret)

if token is not None:
    # Base URL for Spotify API
    base_url = 'https://api.spotify.com/v1/audio-features/'

    # Headers for the API request
    headers = {
        'Authorization': f'Bearer {token}'
    }

    # Loop through each song
    for index, row in songs_df.iterrows(): #songs_df.iloc[2000:].iterrows():
        song_id = row['song_id']
        try:
            response = requests.get(base_url + song_id, headers=headers)
            if response.status_code == 429:
                # Handle rate limiting by waiting for the recommended time
                retry_after = int(response.headers.get('Retry-After', 5))  # Default to 5 seconds if Retry-After is not provided
                print(f"Rate limited. Waiting for {retry_after} seconds...")
                time.sleep(retry_after)
                response = requests.get(base_url + song_id, headers=headers)  # Retry the request
            response.raise_for_status()
            audio_features = response.json()

            # Store the valence and energy in the DataFrame
            if 'valence' in audio_features and 'energy' in audio_features:
                songs_df.at[index, 'valence'] = audio_features['valence']
                songs_df.at[index, 'energy'] = audio_features['energy']
            else:
                print(f"Audio features not found for song {song_id}")

        except requests.exceptions.RequestException as e:
            print(f"Error processing song {song_id}: {e}")

        # Introduce a delay between API requests
        time.sleep(1)

In [21]:
songs_df.head()

,song_id,song_name,artist_id,artist_name,popularity,playlist,lyrics_url,lyrics,language,valence,energy
0,1Eb90nmqTrxylKFhcUzW8P,(Everything I Do) I Do It For You,3Z02hBLubJxuFJfhacLSDc,Bryan Adams,78,['2ubOXCCHl3ysPpRAKAwDwA'],https://genius.com/Bryan-adams-everything-i-do...,[Verse 1]\nLook into my eyes\nYou will see\nWh...,en,0.299,0.36
1,6eBK3edMW7bEzecF1eCezc,(Everything I Do) I Do It For You,3Z02hBLubJxuFJfhacLSDc,Bryan Adams,66,"['37i9dQZF1DX7L1VcqQ1quZ', '37i9dQZF1DX3xgbKZS...",https://genius.com/Bryan-adams-everything-i-do...,[Verse 1]\nLook into my eyes\nYou will see\nWh...,en,0.253,0.365
2,5Y8gFnBzwcMEWa2vmmUABR,(Ghost) Riders In The Sky,72JEwd3EjDEwTLypkFYJn9,The Outlaws,40,['6gKuzkvSdcAL1pCoEdpYyp'],https://genius.com/The-outlaws-southern-rock-b...,An old cowboy went ridin' out one dark and win...,en,0.158,0.948
3,3I1vEQhGwRK7URrTq4BqNl,(Ghost) Riders in the Sky,6kACVPfCOnqzgfEF5ryl0x,Johnny Cash,65,"['1qddPN7vxjhEzkZkFqwLqa', '5xA7C0LJo9jGmqyJlT...",https://genius.com/Johnny-cash-ghost-riders-in...,[Verse 1]\nAn old cowboy went ridin' out one d...,en,0.56,0.668
4,4qHJ2Ap72rtcupZrn7f1do,(I Got A) Stomach Ache,78CBFzwo7wwNaaTYVP5btK,Junior Wells,31,['37i9dQZF1DXd8nB8TGVGGF'],https://genius.com/Junior-wells-i-got-a-stomac...,"Come on, ooh, come on\nCome on, baby\nOoh, I g...",en,0.874,0.443


In [36]:
# Save the modified DataFrame back to a CSV file
songs_df.to_csv('songs_with_audio_features_final.csv', index=False)